In [8]:
import os
os.environ["MANTLE"] = "lattice"
import silica as si
import magma as m
from silica import bits, add, uint
from magma.bit_vector import BitVector
from magma.testing.coroutine import check

@si.coroutine(inputs={"I" : si.Bit})
def Detect111():
    cnt = uint(0, 2)
    I = yield
    while True:
        if (I):
            if (cnt<3):
                cnt = cnt+1
        else:
            cnt = 0
        O = (cnt == 3)
        I = yield O
        
detect111 = si.compile(Detect111(), file_name="build/silica_detect111.py")
with open("build/silica_detect111.py", "r") as magma_file:
    print(magma_file.read())

from magma import *
import os
os.environ["MANTLE"] = os.getenv("MANTLE", "coreir")
from mantle import *
import mantle.common.operator


@cache_definition
def DefineSilicaMux(height, width):
    if "one-hot" == "one-hot":
        if width is None:
            T = Bit
        else:
            T = Bits(width)
        inputs = []
        for i in range(height):
            inputs += [f"I{i}", In(T)]
        class OneHotMux(Circuit):
            name = "SilicaOneHotMux{}{}".format(height, width)
            IO = inputs + ["S", In(Bits(height)), "O", Out(T)]
            @classmethod
            def definition(io):
                or_ = Or(height, width)
                wire(io.O, or_.O)
                for i in range(height):
                    and_ = And(2, width)
                    wire(and_.I0, getattr(io, f"I{i}"))
                    if width is not None:
                        for j in range(width):
                            wire(and_.I1[j], io.S[i])
                    else:
   

In [9]:
!magma -o coreir -m coreir -t Detect111 build/silica_detect111.py
!coreir -i build/silica_detect111.json -o build/silica_detect111.v
!yosys -p 'synth_ice40 -top Detect111 -blif build/silica_detect111.blif' build/silica_detect111.v | grep -A 14 "2.27. Printing statistics."
!arachne-pnr -d 1k -o build/silica_detect111.txt build/silica_detect111.blif
!icetime -tmd hx1k build/silica_detect111.txt  | grep -B 2 "Total path delay"

In Run Generators
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!
ALREADY ADDED CONNECTION!

Modified?: Yes
2.27. Printing statistics.

=== Detect111 ===

   Number of wires:                193
   Number of wire bits:            263
   Number of public wires:         191
   Number of public wire bits:     261
   Number of memories:               0
   Number of memory bits:            0
   Number of processes:              0
   Number of cells:                  8
     SB_DFF                          4
     SB_LUT4                         4

seed: 1
device: 1k
read_chipdb +/share/arachne-pnr/chipdb-1k.bin...
  supported packages: cb121, cb132, cb81, cm121, cm36, cm49, cm81, qn84, swg16tr, tq144, vq100
read_blif build/silica_detect111.blif...
prune...
instantiate_io...
pack...

After packing:
IOs          3 / 96
GBs          0 / 8


In [3]:
from magma.testing.coroutine import check

detect = Detect111()
inputs =  list(map(bool, [1,1,0,1,1,1,0,1,0,1,1,1,1,1,1]))
outputs = list(map(bool, [0,0,0,0,0,1,0,0,0,0,0,1,1,1,1]))
for i, o in zip(inputs, outputs):
    assert o == detect.send(i)

   
@si.coroutine
def inputs_generator(inputs):
    while True:
        for i in inputs:
            I = i
            yield I
 
    
check(detect111, Detect111(), len(inputs), inputs_generator(inputs))
print("Passed!")

Passed!


In [4]:
!yosys -p 'synth_ice40 -top detect111 -blif build/verilog_detect111.blif' ../verilog/detect111.v | grep -A 20 "2.27. Printing statistics."
!arachne-pnr -d 1k -o build/verilog_detect111.txt build/verilog_detect111.blif
!icetime -tmd hx1k build/verilog_detect111.txt | grep -B 2 "Total path delay"

2.27. Printing statistics.

=== detect111 ===

   Number of wires:                  8
   Number of wire bits:              9
   Number of public wires:           4
   Number of public wire bits:       5
   Number of memories:               0
   Number of memory bits:            0
   Number of processes:              0
   Number of cells:                  7
     SB_DFFESR                       2
     SB_LUT4                         5

2.28. Executing CHECK pass (checking for obvious problems).
checking module detect111..
found and reported 0 problems.

2.29. Executing BLIF backend.

seed: 1
device: 1k
read_chipdb +/share/arachne-pnr/chipdb-1k.bin...
  supported packages: cb121, cb132, cb81, cm121, cm36, cm49, cm81, qn84, swg16tr, tq144, vq100
read_blif build/verilog_detect111.blif...
prune...
instantiate_io...
pack...

After packing:
IOs          3 / 96
GBs          0 / 8
  GB_IOs     0 / 8
LCs          5 / 1280
  DFF        2
  CARRY      0
  CARRY, DFF 0
  DFF PASS   0
  CARRY PASS 0


In [5]:
import magma
from magma.testing.verilator import compile as compileverilator
from magma.testing.verilator import run_verilator_test
def check_verilog(circ, circ_sim, num_cycles, inputs_generator=None):
    test_vectors = magma.testing.coroutine.testvectors(circ, circ_sim, num_cycles,
            inputs_generator if inputs_generator else None)

    name = "detect111"
    with open(f"build/sim_{name}_verilator.cpp", "w") as verilator_harness:
        verilator_harness.write(f'''\
#include "V{name}.h"
#include "verilated.h"
#include <cassert>
#include <iostream>

void check(const char* port, int a, int b, int cycle) {{
    if (!(a == b)) {{
        std::cerr << \"Got      : \" << a << std::endl;
        std::cerr << \"Expected : \" << b << std::endl;
        std::cerr << \"Cycle    : \" << cycle << std::endl;
        std::cerr << \"Port     : \" << port << std::endl;
        exit(1);
    }}
}}

int main(int argc, char **argv, char **env) {{
    Verilated::commandArgs(argc, argv);
    V{name}* top = new V{name};
''')
    
        cycle = 0
        for inputs, outputs in zip(test_vectors[0], test_vectors[1]):
            for port_name, value in inputs.items():
                value = int(value)
                verilator_harness.write("    top->{} = {};\n".format(port_name, value))

            verilator_harness.write("    top->CLK = 0;\n")
            verilator_harness.write("    top->eval();\n")
            verilator_harness.write("    top->CLK = 1;\n")
            verilator_harness.write("    top->eval();\n")
            for port_name, value in outputs.items():
                value = int(value)
                verilator_harness.write("    top->eval();\n")
                verilator_harness.write("    check(\"{port_name}\", top->{port_name}, {expected}, {cycle});\n".format(port_name=port_name, expected=value, cycle=cycle))

            cycle += 1
        verilator_harness.write("}\n")

    run_verilator_test(
        "detect111",
        f"sim_{name}_verilator",
        name,
        "-I../../verilog"
    )



check_verilog(detect111, Detect111(), len(inputs), inputs_generator(inputs))
print("Passed!")

Passed!
